In [ ]:
import zarr
import numpy as np
import anndata
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import dask
from pyseq import image_analysis as ia
import warnings
warnings.filterwarnings('ignore')
from dask.distributed import Client
import numba
import joblib
import skimage
import time
from joblib import Parallel, delayed
from joblib import parallel_backend
from dask.distributed import progress
%run /gpfs/commons/home/jsingh/util_pyseq.ipynb

In [ ]:
#Loading Data
im = ia.get_HiSeqImages(image_path = 'zarrs/m387ntga2.zarr')
labels = skimage.io.imread('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')
labels = dask.array.asarray(labels)

In [ ]:
#format = one_z_plane_obj_step_channel_cycle
array_object_list = []
name_object_list = []
for i in im.im['channel'].values:
    for j in im.im['cycle'].values:
        #for k #in im.im['obj_step'].values[1]:
            k = 8028
            name = "one_x_plane_"+str(i)+"_"+str(j)+"_"+str(k)
            name_object_list.append(name)
            nme = im.im.sel(obj_step = k, cycle=j, channel = i)
            array_object_list.append(nme)
        

In [ ]:
############ INITIATING CLUSTER ####################

In [ ]:
def get_cluster(queue_name = 'pe2', log_dir=None):
    """ Make dask cluster w/ workers = 2 cores, 32 G mem, and 1 hr wall time.

        return cluster, client
    """

    if log_dir is None:
        log_dir = join(getcwd(),'dask_logs')
        makedirs(log_dir, exist_ok=True)

    cluster = SLURMCluster(
                queue = queue_name, 
                cores = 12 ,
                memory = '128G',
                walltime='1:00:00',
                log_directory=log_dir)
                #extra=["--lifetime", "55m", "--lifetime-stagger", "4m"])
    client = Client(cluster)

    return cluster, client

cluster, client = get_cluster()

In [ ]:
def scale_cluster: 
    cluster.scale(15)
    return cluster.dashboard_link

In [ ]:
#Or specify cores or memory directly

In [ ]:
plane = array_object_list[0] #sample plane 

In [ ]:
#Way 1: Using Data As Is (No Persit) with Dask Futures
plane = array_object_list[0]
def get_pixels(lab):
    m = plane.values[labels == lab+1].mean()
    return m

futures = client.map(get_pixels, range(5))
progress(futures)

result_list = np.zeros(20)
for i in range(len(futures)):
    result_list[i] = futures[i].result()
    
    

In [ ]:
#Way 2: Using with Dask Persist with Dask Futures
plane_persisted = array_object_list[0].persist()
def get_pixels(lab):
    m = plane_persisted.values[labels == lab+1].mean()
    return m
from dask.distributed import progress
futures = client.map(get_pixels, range(5))
progress(futures)


result_list = np.zeros(20)
for i in range(len(futures)):
    result_list[i] = futures[i].result()

In [ ]:
%%time
#Way 3: Computing the same using dask compute without persisting Data
def get_pixels(lab):
    m = plane.values[labels == lab+1].mean()
    return m

with parallel_backend('dask',scheduler_host=cluster.scheduler._address):
    results = Parallel(n_jobs=-1)(delayed(get_pixels_persisted)(lab) for lab in range(100))

In [ ]:
%%time
#Way 4: Computing way 3 with persist
def get_pixels(lab):
    m = plane_persisted.values[labels == lab+1].mean()
    return m

with parallel_backend('dask',scheduler_host=cluster.scheduler._address):
    results = Parallel(n_jobs=-1)(delayed(get_pixels_persisted)(lab) for lab in range(100))

In [ ]:
dk = dask.array.asarray(plane.values)

In [ ]:
#Way 5: Dask Masked Array
dk = dask.array.asarray(plane.values)
def get_pixels(lab):
    m = dask.array.ma.masked_array(dk,np.squeeze([labels == lab]))
    mv = dask.array.mean(m)
    return mv

futures = client.map(get_pixels, range(5))

result_list = np.zeros(20)
for i in range(len(futures)):
    result_list[i] = futures[i].result()
    
    


In [ ]:
#Way 6: Dask Masked Array With Persist 
dk = dask.array.asarray(persisted_plane.values)
def get_pixels(lab):
    m = dask.array.ma.masked_array(dk,np.squeeze([labels == lab]))
    mv = dask.array.mean(m)
    return mv

futures = client.map(get_pixels, range(5))

result_list = np.zeros(20)
for i in range(len(futures)):
    result_list[i] = futures[i].result()
    



In [ ]:
#Way 5 using joblib 
def get_pixels(lab):
    m = dask.array.ma.masked_array(dk,np.squeeze([labels == lab]))
    mv = dask.array.mean(m)
    return mv

with parallel_backend('dask',scheduler_host=cluster.scheduler._address):
    results = Parallel(n_jobs=-1)(delayed(get_pixels)(lab) for lab in range(100))







In [ ]:
#Way x : Dask image functions
import dask_image

index = list(range(11232))
del index[0]

ar = dask_image.ndmeasure.labeled_comprehension(image = plane, 
                                                label_image = labels, index = index, func = dask_image.ndmeasure.mean,
                                                out_dtype = float, default = None, pass_positions = False)
v = ar.compute()

In [ ]:
#ADJUSTING TIMEOUT

In [ ]:
client.close()
cluster.close()